# 2DFS-999

Updating the dataset: G039050100000

* From the count-rate plots, we know that all channels have pointing problems, so we scale them to match the COS spectrum of this star.  
* Since their exposure times are nearly identical, we assume that the scale factor for LiF 2A is appropriate for LiF 2B and scale the other channels to match.  
* The SiC 1 channels have background-subtraction problems, so we do not use them.

COMMENT File updated 19 October 2022 <br>
COMMENT This NVO file has been reconstructed from the corresponding ALL file.<br>
COMMENT All segments have been scaled to match COS spectrum of this star.<br>
COMMENT SiC 1 channel has problems and is not used.

In [ ]:
%matplotlib inline
import matplotlib

import numpy as np

from astropy import units as u
from astropy.convolution import convolve, Box1DKernel
from astropy.io import fits
from astropy.visualization import quantity_support
from matplotlib import pyplot as plt

from specutils import Spectrum1D
from specutils.spectra import SpectralRegion
from specutils.manipulation import box_smooth, extract_region

# Specify plot parameters.
quantity_support()  # to put units on the axes below 
matplotlib.rcParams['figure.figsize'] = [15, 5]

# Wavelength per channel (pixel) is always 0.013 A.
WPC = 0.013

# Normalize all spectra, so fluxes are of order unity.
norm = 1E13

In [ ]:
# Function to convert FITS array into spectrum object.

def make_spectrum (specdata):
    lamb = specdata['wave'] * u.AA 
    flux = specdata['flux'] * norm * u.Unit('erg cm-2 s-1 AA-1') 
    return Spectrum1D(spectral_axis=lamb, flux=flux)

In [ ]:
# Function to compute offset in pixels between two spectra

def compute_shift(specdata, refdata, wmin, wmax):
    wave_spec  = specdata['wave']
    flux_spec  = specdata['flux']
    error_spec = specdata['error']
    wave_ref   =  refdata['wave']
    flux_ref   =  refdata['flux']
    
    # Compute scale factor between data and reference spectra.
    g = np.where((wave_spec > wmin) & (wave_spec < wmax))
    h = np.where((wave_ref  > wmin) & (wave_ref  < wmax))
    spec_mean = np.mean(flux_spec[g])
    ref_mean  = np.mean(flux_ref[h])
    scale = ref_mean / spec_mean

    # Smooth data and error arrays.
    flux_spec  = convolve(flux_spec,  Box1DKernel(17))
    error_spec = convolve(error_spec, Box1DKernel(17))
    flux_ref   = convolve(flux_ref,   Box1DKernel(17))
    
    # Rescale spectrum to match reference.
    flux_spec *= scale
    error_spec *= scale
    
    # Compute minimum value of chi-squared.
    chisq = np.zeros(40)
    for i in range(40):
        j = i - 20
        chisq[i] = np.sum(((flux_spec[g[0]+j] - flux_ref[h]) / error_spec[g[0]+j])**2)
    
    # We have computed this offset by shifting the flux array, but will 
    # use it by shifting the wavelength array, so must multiply by -1.
    return 20 - np.argmin(chisq)

In [ ]:
# Read keywords from file header.

filename = 'g039050100000all4ttagfcal.fit'
f = fits.open(filename)
print ('Target:  ', f[0].header['TARGNAME'])
print ('Aperture:', f[0].header['APERTURE'])
print ('Guider:  ', f[0].header['FESCENT'])
print ('CHANL OBSTIME COMBMETH')
for i in range(1,9): print (f[i].header['EXTNAME'], f[i].header['OBSTIME'], f[i].header['COMBMETH'])
    
# LiF 2A is the guide channel.

In [ ]:
# Read all eight spectra from FUSE *all* file.

lif1a_data = f['1alif'].data 
lif1b_data = f['1blif'].data
lif2b_data = f['2blif'].data
lif2a_data = f['2alif'].data
sic1a_data = f['1asic'].data
sic1b_data = f['1bsic'].data
sic2b_data = f['2bsic'].data
sic2a_data = f['2asic'].data
f.close() 

In [ ]:
# Convert FITS arrays into spectrum objects.

lif1a = make_spectrum (lif1a_data)
lif1b = make_spectrum (lif1b_data)
lif2a = make_spectrum (lif2a_data)
lif2b = make_spectrum (lif2b_data)
sic1a = make_spectrum (sic1a_data)
sic1b = make_spectrum (sic1b_data)
sic2a = make_spectrum (sic2a_data)
sic2b = make_spectrum (sic2b_data)

In [ ]:
# Smooth the spectral arrays.

from specutils.manipulation import (box_smooth)

lif1a_bsmooth = box_smooth(lif1a, width=15)
lif1b_bsmooth = box_smooth(lif1b, width=15)
lif2a_bsmooth = box_smooth(lif2a, width=15)
lif2b_bsmooth = box_smooth(lif2b, width=15)
sic1a_bsmooth = box_smooth(sic1a, width=15)
sic1b_bsmooth = box_smooth(sic1b, width=15)
sic2a_bsmooth = box_smooth(sic2a, width=15)
sic2b_bsmooth = box_smooth(sic2b, width=15)

In [ ]:
# From the count-rate plots, we know that all channels have pointing problems,
# so we scale the FUSE data to match a COS spectrum of the same object.

# First examine the long-wavelength spectra.

f, ax = plt.subplots()  
ax.step(lif2a_bsmooth.spectral_axis, lif2a_bsmooth.flux, label='LiF 2A') 
ax.step(lif1b_bsmooth.spectral_axis, lif1b_bsmooth.flux, label='LiF 1B') 
ax.legend()
ax.set_ylim([0,4])

# LiF 1B suffers from a worm, but is OK at long wavelengths.

In [ ]:
# Now compare with a COS spectrum of the same star.

filename = 'lehg3c010_x1dsum.fits'
cos = Spectrum1D.read(filename, format="HST/COS")
print('Target:', fits.getval(filename, ext=0, keyword='TARGNAME'))
cos_bsmooth = box_smooth(cos, width=15)

f, ax = plt.subplots()
ax.plot(cos_bsmooth.spectral_axis, norm * cos_bsmooth.flux, label='COS')  
ax.step(lif2a_bsmooth.spectral_axis, lif2a_bsmooth.flux, label='LiF 2A') 
ax.step(lif1b_bsmooth.spectral_axis, lif1b_bsmooth.flux, label='LiF 1B') 
ax.legend()
ax.set_xlim([1140,1200])
ax.set_ylim([0,4])

# Both FUSE channels are a bit faint.

In [ ]:
# Rescale both spectra to match COS.

region = SpectralRegion([(1155*u.AA, 1167.5*u.AA),(1170*u.AA, 1175*u.AA)])
sub_cos = extract_region(cos, region, return_single_spectrum=True)
sub_lif2a = extract_region(lif2a, region, return_single_spectrum=True)
sub_lif1b = extract_region(lif1b, region, return_single_spectrum=True)

# Compute ratio of the flux to COS.

mean_cos = norm * sub_cos.mean()
scale_lif2a = mean_cos/sub_lif2a.mean()
print ('Scale LiF 2A by', scale_lif2a)
scale_lif1b = mean_cos/sub_lif1b.mean()
print ('Scale LiF 1B by', scale_lif1b)

In [ ]:
# Scale the spectra.

lif2a_bsmooth *= scale_lif2a
lif1b_bsmooth *= scale_lif1b

In [ ]:
# Plot the result.

f, ax = plt.subplots()
ax.plot(cos_bsmooth.spectral_axis, norm * cos_bsmooth.flux, label='COS')  
ax.step(lif2a_bsmooth.spectral_axis, lif2a_bsmooth.flux, label='LiF 2A') 
ax.step(lif1b_bsmooth.spectral_axis, lif1b_bsmooth.flux, label='LiF 1B') 
ax.legend()
ax.set_xlim([1140,1200])
ax.set_ylim([0,4])

In [ ]:
# Now consider region between 1000 and 1100 A.

f, ax = plt.subplots()  
ax.step(lif1a_bsmooth.spectral_axis, lif1a_bsmooth.flux, label='LiF 1A') 
ax.step(lif2b_bsmooth.spectral_axis, lif2b_bsmooth.flux, label='LiF 2B') 
ax.step(sic1a_bsmooth.spectral_axis, sic1a_bsmooth.flux, label='SiC 1A') 
ax.step(sic2b_bsmooth.spectral_axis, sic2b_bsmooth.flux, label='SiC 2B') 
ax.legend()
# ax.set_xlim([1040, 1090])
ax.set_ylim([-0.5,5])

# The LiF channels match pretty well, the SiC channels less so.

In [ ]:
# LiF 2 is the guide channel, and the LiF 2A and 2B channels have (nearly) identical exposure times.
# We assume that the scale factor for LiF 2A applies to LiF 2B and scale the other channels to match.

# Select a broad spectral region.

region = SpectralRegion(1040*u.AA, 1070*u.AA)
sub_lif1a = extract_region(lif1a, region)
sub_lif2b = extract_region(lif2b, region)
sub_sic1a = extract_region(sic1a, region)
sub_sic2b = extract_region(sic2b, region)

# Compute ratio of all fluxes to LiF 2B (rescaled).

mean = scale_lif2a * sub_lif2b.mean()
scale_lif1a = mean/sub_lif1a.mean()
print ('Scale LiF 1A by', scale_lif1a)
scale_lif2b = scale_lif2a
print ('Scale LiF 2B by', scale_lif2b)

scale_sic1a = mean/sub_sic1a.mean()
print ('Scale SiC 1A by', scale_sic1a)
scale_sic2b = mean/sub_sic2b.mean()
print ('Scale SiC 2B by', scale_sic2b)

In [ ]:
# Scale the spectra.

lif1a *= scale_lif1a
lif2b *= scale_lif2b
sic1a *= scale_sic1a
sic2b *= scale_sic2b

lif1a_bsmooth *= scale_lif1a
lif2b_bsmooth *= scale_lif2b
sic1a_bsmooth *= scale_sic1a
sic2b_bsmooth *= scale_sic2b

In [ ]:
# Replot the rescaled spectra.

f, ax = plt.subplots()  
ax.step(lif1a_bsmooth.spectral_axis, lif1a_bsmooth.flux, label='LiF 1A') 
ax.step(lif2b_bsmooth.spectral_axis, lif2b_bsmooth.flux, label='LiF 2B') 
ax.step(sic1a_bsmooth.spectral_axis, sic1a_bsmooth.flux, label='SiC 1A')
ax.step(sic2b_bsmooth.spectral_axis, sic2b_bsmooth.flux, label='SiC 2B')
ax.legend()
ax.set_xlim([1000, 1040])
ax.set_ylim([-2,5])

# SiC 1A has background-subtraction problems.  Do not use it.

In [ ]:
# At short wavelengths, we must boot-strap our way down.

f, ax = plt.subplots() 
ax.step(lif2b_bsmooth.spectral_axis, lif2b_bsmooth.flux, label='LiF 2B') 
ax.step(sic1b_bsmooth.spectral_axis, sic1b_bsmooth.flux, label='SiC 1B')
ax.step(sic2a_bsmooth.spectral_axis, sic2a_bsmooth.flux, label='SiC 2A')
ax.legend()
ax.set_xlim([900, 1010])
ax.set_ylim([-0.2,4])

# SiC 1B also has background-subtraction problems.  We will not consider it further.

In [ ]:
# Use LiF 2B to rescale SiC 2A.

# Select a spectral region.

region = SpectralRegion(995*u.AA, 1005*u.AA)
sub_lif2b = extract_region(lif2b, region)
sub_sic2a = extract_region(sic2a, region)

# Compute ratio of the flux to LiF 1A.

scale_sic2a = sub_lif2b.mean()/sub_sic2a.mean()
print ('Scale SiC 2A by', scale_sic2a)

# Scale the spectrum.

sic2a_bsmooth *= scale_sic2a

In [ ]:
# Plot the scaled spectra.

f, ax = plt.subplots() 
ax.step(lif2b_bsmooth.spectral_axis, lif2b_bsmooth.flux, label='LiF 2B')
ax.step(sic2a_bsmooth.spectral_axis, sic2a_bsmooth.flux, label='SiC 2A')
ax.legend()
ax.set_xlim([900, 1010])
ax.set_ylim([-0.2,5])

In [ ]:
# Next we compute the shift (in pixels) relative to LiF 2b.

shift_lif1 = compute_shift(lif1a_data, lif2b_data, 1040, 1070)
shift_sic1 = compute_shift(sic1a_data, lif2b_data, 1040, 1070)
shift_sic2 = compute_shift(sic2b_data, lif2b_data, 1040, 1070)

print ('Shift LiF 1 by ', shift_lif1, ' pixels.')
print ('Shift SiC 1 by ', shift_sic1, ' pixels.')
print ('Shift SiC 2 by ', shift_sic2, ' pixels.')

In [ ]:
# Shift all eight spectra.

shift_lif1 *= WPC
shift_sic1 *= WPC
shift_sic2 *= WPC

lif1a_shift = Spectrum1D(spectral_axis=lif1a.spectral_axis + shift_lif1 * u.AA, flux=lif1a.flux)
lif1b_shift = Spectrum1D(spectral_axis=lif1b.spectral_axis + shift_lif1 * u.AA, flux=lif1b.flux)
lif2a_shift = lif2a
lif2b_shift = lif2b
sic1a_shift = Spectrum1D(spectral_axis=sic1a.spectral_axis + shift_sic1 * u.AA, flux=sic1a.flux)
sic1b_shift = Spectrum1D(spectral_axis=sic1b.spectral_axis + shift_sic1 * u.AA, flux=sic1b.flux)
sic2a_shift = Spectrum1D(spectral_axis=sic2a.spectral_axis + shift_sic2 * u.AA, flux=sic2a.flux)
sic2b_shift = Spectrum1D(spectral_axis=sic2b.spectral_axis + shift_sic2 * u.AA, flux=sic2b.flux)

f, ax = plt.subplots()  
ax.step(lif1a_shift.spectral_axis, lif1a_bsmooth.flux, label='LiF 1A') 
ax.step(lif2b_shift.spectral_axis, lif2b_bsmooth.flux, label='LiF 2B')
ax.step(sic1a_shift.spectral_axis, sic1a_bsmooth.flux, label='SiC 1A') 
ax.step(sic2b_shift.spectral_axis, sic2b_bsmooth.flux, label='SiC 2B')
ax.legend()
ax.set_xlim([1040,1070])
ax.set_ylim([0,5])

In [ ]:
# We reassemble the NVO file.  We omit the SiC 1 channel, as it has problems.

filename = 'g039050100000nvo4ttagfcal.fit'
f = fits.open(filename)
hdr = f[0].header
data = f[1].data 

data['flux'] = 0.
data['error'] = 0.

# Scale SiC 2A
sic2a_wave = sic2a_shift.spectral_axis.value
g = np.where((data['wave'] > 917.5) & (data['wave'] < 998))
h = np.where((sic2a_wave > 917.5) & (sic2a_wave < 998))
data['flux'][g] = float(scale_sic2a) * sic2a_data['flux'][h]
data['error'][g] = float(scale_sic2a) * sic2a_data['error'][h]

# Scale LiF 1A
lif1a_wave = lif1a_shift.spectral_axis.value
g = np.where((data['wave'] > 998) & (data['wave'] < 1080))
h = np.where((lif1a_wave > 998) & (lif1a_wave < 1080))
data['flux'][g] = float(scale_lif1a) * lif1a_data['flux'][h]
data['error'][g] = float(scale_lif1a) * lif1a_data['error'][h]

# Scale SiC 2B 
sic2b_wave = sic2b_shift.spectral_axis.value
g = np.where((data['wave'] > 1080) & (data['wave'] < 1090))
h = np.where((sic2b_wave > 1080) & (sic2b_wave < 1090))
data['flux'][g] = float(scale_sic2b) * sic2b_data['flux'][h]
data['error'][g] = float(scale_sic2b) * sic2b_data['error'][h]

# Scale LiF 2A
lif2a_wave = lif2a_shift.spectral_axis.value
g = np.where((data['wave'] > 1090) & (data['wave'] < 1180))
h = np.where((lif2a_wave > 1090) & (lif2a_wave < 1180))
data['flux'][g] = float(scale_lif2a) * lif2a_data['flux'][h]
data['error'][g] = float(scale_lif2a) * lif2a_data['error'][h]

# Scale LiF 1B
lif1b_wave = lif1b_shift.spectral_axis.value
h = np.where((lif1b_wave > 1180) & (lif1b_wave < 1190.01))
g = np.where((data['wave'] > 1180) & (data['wave'] < np.max(lif1b_wave[h]) + 0.01))
data['flux'][g] = float(scale_lif1b) * lif1b_data['flux'][h]
data['error'][g] = float(scale_lif1b) * lif1b_data['error'][h]

hdr['comment'] = ''
hdr['comment'] = 'File updated 19 October 2022'
hdr['comment'] = 'This NVO file has been reconstructed from the corresponding ALL file.'
hdr['comment'] = 'All segments have been scaled to match COS spectrum of this star.'
hdr['comment'] = 'SiC 1 channel has problems and is not used.'

f.writeto('level0_g039050100000nvo4ttagfcal_vo.fit', overwrite=True)
f.close()

In [ ]:
# Compare old and new versions of NVO file.

filename = 'g039050100000nvo4ttagfcal.fit'
f = fits.open(filename)
old = f[1].data 
f.close()

filename = 'level0_g039050100000nvo4ttagfcal_vo.fit'
f = fits.open(filename)
hdr = f[0].header
new = f[1].data 
f.close()

print (hdr['comment'])

f, (ax1, ax2) = plt.subplots(2, 1, sharey=True)  
ax1.step(new['wave'], new['flux'], label='NEW FLUX')
ax1.step(old['wave'], old['flux'], label='OLD FLUX') 
ax2.step(new['wave'], new['flux'], label='NEW FLUX')
ax2.step(old['wave'], old['flux'], label='OLD FLUX') 
ax1.legend()
ax1.set_xlim([900, 1050])
ax2.set_xlim([1050, 1190])
ax1.set_ylim([0,1E-12])

In [ ]:
# Let's look more closely at the regions sampled only by the SiC channels.

f, ax = plt.subplots()  
ax.step(new['wave'], new['flux'], label='NEW FLUX')
ax.step(old['wave'], old['flux'], label='OLD FLUX') 
ax.legend()
ax.set_xlim([1070,1095])
ax.set_ylim([-1E-13,1E-12])

# Seems reasonable.